In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import math

In [ ]:
from google.colab import files
uploaded = files.upload../input/hashcode-drone-delivery/busy_day.in()

In [ ]:
with open('../input/hashcode-drone-delivery/busy_day.in') as file:
    line_list = file.read().splitlines()

In [ ]:
len(line_list)

In [ ]:
line_list[:4]

1: parameters of the simulation

2: weights of the products available for orders

3: warehouses and availability of individual product types

4: customer orders

**Simulation**


First section means parameters of the simulation

In [ ]:
first_section = line_list[0].split()
first_section

400: rows - number of rows in the area of the simulation (min:1, max:10000)

600: columns - number of rows in the area of the simulation (min:1, max:10000)

30: drons - number of drones available (min: 1, max: 1000)

112993: turns - this is the simulation of drone delivery, it proceeds in 112993 turns. (min: 1, max: 1000000)

200 : max payload - maximum load of a drone (min: 1, max: 10000)

**Product**

In [ ]:
second_section = line_list[1].split()
second_section

Answer - There are 400 different product types.

In [ ]:
third_section = line_list[2].split()
len(third_section)

Answer- 400: length of product types weights



In [ ]:
third_section[:10]

**Warehouse**

In [ ]:
fourth_section = line_list[3].split()
fourth_section

Answer- There are 10 warehouses.

In [ ]:
fifth_section = line_list[4].split()
fifth_section

Answer -First warehouse is loacated at 113, 179.

In [ ]:
sixth_section = line_list[5].split()
len(sixth_section)

Answer-First warehouse stores 400 items. It can contains 0, so we need to check this list.



In [ ]:
seventh_section = line_list[6].split()
seventh_section

Answer-Second warehouse is loacated at 234, 599.

In [ ]:
eightth_section = line_list[7].split()
len(eightth_section)

we can check 10 + 10 sections for warehouses. Later, we will look whole list.



**Order**

In [ ]:
order_section = line_list[24].split()
order_section

There are 1250 orders.

In [ ]:
first_order_delivered = line_list[25].split()
first_order_delivered

First order to be delivered to 340, 371.

In [ ]:
first_order_items = line_list[26].split()
first_order_items

First order contains 8 items.





Simillar to sections for warehouses, we can check 1250 + 1250.

In [ ]:
len(line_list)

**Get Dataframe for each section**


In [ ]:
# Products
weights = line_list[2].split()
products_df = pd.DataFrame({'weight': weights})

wh_count = int(line_list[3])
wh_endline = (wh_count*2)+4

wh_invs = line_list[5:wh_endline+1:2]
for i, wh_inv in enumerate(wh_invs):
    products_df[f'wh{i}_inv'] = wh_inv.split()

products_df = products_df.astype(int)

# Warehouses
wh_locs = line_list[4:wh_endline:2]
wh_rows = [wl.split()[0] for wl in wh_locs]
wh_cols = [wl.split()[1] for wl in wh_locs]

warehouse_df = pd.DataFrame({'wh_row': wh_rows, 'wh_col': wh_cols}).astype(np.uint16)

# Orders
order_locs = line_list[wh_endline+1::3]
o_rows = [ol.split()[0] for ol in order_locs]
o_cols = [ol.split()[1] for ol in order_locs]

orders_df = pd.DataFrame({'row': o_rows, 'col': o_cols})

orders_df[orders_df.duplicated(keep=False)].sort_values('row')

orders_df['product_count'] = line_list[wh_endline+2::3]

order_array = np.zeros((len(orders_df), len(products_df)), dtype=np.uint16)
orders = line_list[wh_endline+3::3]
for i,ord in enumerate(orders):
    products = [int(prod) for prod in ord.split()]
    order_array[i, products] = 1

df = pd.DataFrame(data=order_array, columns=['p_'+ str(i) for i in range(400)], 
                    index=orders_df.index)

orders_df = orders_df.astype(np.uint16).join(df)

In [ ]:
products_df.head(10)

In [ ]:
warehouse_df.head(10)

In [ ]:
orders_df.head(10)

**Cheking lenght of order list in data**

In [ ]:
len(orders_df)

**Visualization**

In [ ]:
chart_opts = {'width': 500,
              'xlabel': "Total Demand",
              'ylabel': "Count of Products"}

import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

counts = orders_df.product_count \
                  .value_counts() \
                  .sort_index() \
                  .reset_index()
hv.Bars(counts).opts(**chart_opts)

In [ ]:
chart={'width': 600, 'xlabel':'surplus'}

import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

supply = products_df.drop(columns='weight').sum(axis=1)
#print(supply)

demand = orders_df.loc[:, orders_df.columns.str.contains("p_")].sum()
#print(demand)

surplus = supply.to_numpy() - demand.to_numpy()
#print(np.amin(surplus))


edges , freqs = np.histogram(surplus, 20)
#print(freqs, edges)
hv.Histogram((edges , freqs)).opts(**chart)

In [ ]:
chart_opts = {'width': 500,
              'xlabel': "Warehouse",
              'ylabel': "Total Inventory",
              'yticks': list(range(0,1801,200))}

import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

total_prods = products_df.loc[:, products_df.columns.str.contains("wh")].sum()
hv.Bars(total_prods.value_counts().index).opts(**chart_opts)

In [ ]:
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

hv.Distribution(products_df.weight).opts(width=500)

In [ ]:
chart_opts = dict(width=600, height=400, alpha=0.7)

import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

customers = hv.Points(orders_df, kdims = ['col', 'row']).opts(**chart_opts)
warehouses = hv.Points(warehouse_df, kdims = ['wh_col', 'wh_row']).opts(size=8, **chart_opts)
customers * warehouses

In [ ]:
def list_lines(file_name):
    """Returns a list of integer lists."""
    with open(file_name) as file:
        lines = file.read().splitlines()
    line_list = [[int(n) for  n in ll.split()] for ll in lines]
    return line_list

def list_lines(file_name):
    """Returns a list of integer lists."""
    with open(file_name) as file:
        lines = file.read().splitlines()
    line_list = [[int(n) for  n in ll.split()] for ll in lines]
    return line_list


def set_params(line_list):
    top_line = line_list[0]
    params = {'DRONE_COUNT': top_line[2],
              'WT_CAP': top_line[4],
              'END_TIME': top_line[3],
              }
    return params


def find_wh_lines(line_list):
    """Provides the dividing line between warehouse and
    order sections in the line list."""
    wh_count = line_list[3][0]
    wh_endline = (wh_count*2)+4
    return wh_endline


def get_weights(line_list):
    weights = np.array(line_list[2])
    return weights.astype(np.int16)


def get_inventories(line_list):
    """Returns a 2-d array of P products by W warehouses."""
    wh_endline = find_wh_lines(line_list)
    invs = line_list[5:wh_endline+1:2]
    supply = np.array(invs).transpose()
    return supply.astype(np.int16)


def get_orders(line_list):
    """Returns a 2-d array of P products by C orders."""
    wh_endline = find_wh_lines(line_list)
    demand = np.zeros((line_list[1][0], line_list[wh_endline][0]),
                            dtype=np.int16)
    orders = line_list[wh_endline+3::3]
    for i,ord in enumerate(orders):
        for prod in ord:
            demand[prod, i] += 1
    return demand.astype(np.int16)


def get_locs(line_list):
    wh_endline = find_wh_lines(line_list)
    wh_locs = np.array(line_list[4:wh_endline:2])
    cust_locs = np.array(line_list[wh_endline+1::3])
    return wh_locs.astype(np.int16), cust_locs.astype(np.int16)


In [ ]:
files = ['/content/busy_day.in']
line_list = list_lines(files[0])

params = set_params(line_list)
print(params)
supply = get_inventories(line_list)
demand = get_orders(line_list)
wh_locs, cust_locs = get_locs(line_list)
weights = get_weights(line_list)

In [ ]:
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

freqs, edges = np.histogram(weights, 20)
wt_prod = hv.Histogram((edges, freqs)).options(xlabel="product weights"
                                               ,width=250)

order_weights = (weights.reshape(weights.size, -1)* demand) \
                    .sum(axis=0)
freqs, edges = np.histogram(order_weights, 20)
wt_orders = hv.Histogram((edges, freqs)).options(xlabel="order weights",
                                                 width=400)

surplus = hv.Curve(supply.sum(axis=1) - demand.sum(axis=1)).options(width=500,
                                            xlabel='product', ylabel='surplus')

customers = hv.Points(np.fliplr(cust_locs)).options(width=600, height=400)
warehouses = hv.Points(np.fliplr(wh_locs)).options(size=8, alpha=0.5)

display(hv.Layout(wt_prod+wt_orders).options(shared_axes=False), surplus, 
            customers*warehouses)